# Pose Unposed Cameras

In [2]:
# load environment

# Load Cameras

import os
import sys
import numpy as np
import json

sys.path.append('..')
from configs.arguments import get_config_dict
from utils.multiview_utils import Camera, Calibration, MultiviewVids
from utils.io_utils import write_json, load_json
from utils.metadata_utils import get_cam_names
from utils.coordinate_utils import update_reconstruction, point_in_polygon, project_to_ground_plane_cv2
from utils.plot_utils import rotation_matrix, perp

from scipy.spatial.transform import Rotation as R
from skspatial.objects import Point, Vector, Plane, Points, Line
import pyransac3d as pyrsc

import copy
import cv2
import ipywidgets as widgets
import ipympl
import matplotlib.pyplot as plt
%matplotlib widget




arv_copy = sys.argv
sys.argv = ['pop']
sys.argv.append('-cfg')
config_path = os.path.abspath('../../project_config.yaml')
sys.argv.append(config_path)
sys.argv.append('-dr')
root_path = os.path.abspath('../../data/')
sys.argv.append(root_path)
sys.argv.append('-l')
sys.argv.append('info')

# load arguments from the arg parser
config = get_config_dict()
data_root = config["main"]["data_root"]
calib_dir = os.path.join(data_root, '0-calibration', 'calibs')
video_dir = os.path.join(data_root, 'raw_data', 'footage')
# reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'undistorted', 'reconstruction.json')
# omni_tag = '360'


# reconstruction_dir = os.path.join(data_root, '0-calibration', 'opensfm', 'reconstruction.json')
# if not os.path.exists(reconstruction_dir):
#     print( "Reconstruction not found. Please run 0-calibration/2-extrinsics.py first.")
    

11:38:12.371 - [...calibration\..\configs\arguments.py:093] - DEBUG    - No training config file specified, using default config in data_root/2-training/train_config.yaml


## Select interval to examine


In [ ]:
# load interval json
interval_json = os.path.join(data_root, '0-calibration', 'calibs', 'interval.json')

# select_interval 
interval_idx = 0
interval_json = load_json(interval_json)
interval = interval_json[interval_json.keys()[interval_idx]]

# load cameras
cam_names = interval['cameras']

# load interval reconstruction
reconstruction = load_json(reconstruction_dir / interval_json.keys()[interval_idx])

# load calibrated cameras
calibrated_cameras = {}
for cam_name in cam_names:
    calib_path = os.path.join(calib_dir, cam_name + '.json')
    calib = load_json(calib_path)
    calibrated_cameras[cam_name] = calib


# load unposed cameras
unposed_cameras = {}
for cam_name in cam_names:
    if cam_name not in reconstruction['cameras']:
        unposed_cameras[cam_name] = calibrated_cameras[cam_name]

In [57]:
# dropdown for selecting the interval


intervals = os.listdir(os.path.join(data_root, '0-calibration'))

interval_dropdown = widgets.Dropdown(
    options=intervals,
    description='Interval:',
    disabled=False,
    value='opensfm'
)

# load calibrated cameras
posed_cameras = []
unposed_cameras = []
posed_images = []
unposed_images = []

display(interval_dropdown)

def update(*args):
    print(interval_dropdown.value)

    interval = interval_dropdown.value

    
    reconstruction_path = os.path.join(data_root, '0-calibration', interval, 'undistorted', 'reconstruction.json')
    images_path = os.path.join(data_root, '0-calibration', interval, 'undistorted', 'images')

    if not os.path.exists(reconstruction_path):
        print( "Reconstruction not found. Please run 0-calibration/2-extrinsics.py for this interval first.")
        return
    reconstruction = load_json(reconstruction_path)[0]

    cam_names = reconstruction['cameras'].keys()

    
    for cam_name in cam_names:
        try:
            camera_name = cam_name.split(' ')[1]
        except:
            camera_name = cam_name.split('_')[0]

        print(camera_name, reconstruction['shots'].keys())
        for shot in reconstruction['shots'].keys():
            if camera_name + '_' in shot:
                posed_cameras.append(camera_name)
                break

        if camera_name not in posed_cameras:
            unposed_cameras.append(cam_name.split(' ')[1])

    if len(unposed_cameras) == 0:
        print('All cameras for this interval are posed already.')

    else:
        print('Unposed cameras for this interval:', unposed_cameras)

interval_dropdown.observe(update, 'value')



Dropdown(description='Interval:', index=4, options=('calibs', 'calibs_old', 'first_frame.json', 'frames', 'ope…

frames
Reconstruction not found. Please run 0-calibration/2-extrinsics.py for this interval first.
opensfm
cam1 dict_keys(['cam360_1768.jpg_perspective_view_bottom.jpg', 'cam360_1768.jpg_perspective_view_front.jpg', 'cam360_1872.jpg_perspective_view_bottom.jpg', 'cam360_1872.jpg_perspective_view_right.jpg', 'cam360_1872.jpg_perspective_view_left.jpg', 'cam360_1355.jpg_perspective_view_top.jpg', 'cam360_1975.jpg_perspective_view_top.jpg', 'cam360_1975.jpg_perspective_view_back.jpg', 'cam360_2182.jpg_perspective_view_bottom.jpg', 'cam360_2182.jpg_perspective_view_back.jpg', 'cam360_2182.jpg_perspective_view_front.jpg', 'cam360_2182.jpg_perspective_view_right.jpg', 'cam360_1665.jpg_perspective_view_right.jpg', 'cam360_1665.jpg_perspective_view_back.jpg', 'cam360_1665.jpg_perspective_view_left.jpg', 'cam360_1665.jpg_perspective_view_front.jpg', 'cam360_1406.jpg_perspective_view_bottom.jpg', 'cam360_1406.jpg_perspective_view_top.jpg', 'cam360_1406.jpg_perspective_view_right.jpg', 'cam360_14

## Annotate Matched Keypoints

In [41]:
# load images

import cv2



opensfm


In [52]:
unposed_selector = widgets.SelectMultiple(
    options=unposed_cameras,
    value=[],
    description='Unposed Cameras',
    disabled=False
)

posed_selector = widgets.SelectMultiple(
    options=posed_cameras,
    value=[],
    description='Posed Cameras',
    disabled=False
)

unposed_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Unposed Image:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
posed_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Posed Image:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(unposed_selector)
display(posed_selector)


def set_img_sliders(*args):
    # clear previous sliders
    # unposed_slider.close()

    # posed_slider.close()

    # update sliders with new values
    unposed_slider.max = len(unposed_cameras[unposed_selector.value])
    posed_slider.max = len(posed_cameras[posed_selector.value])
    
    
    
    display(unposed_slider)
    display(posed_slider)


def render(*args):
    print(unposed_selector.value)
    print(posed_selector.value)
    print(interval_dropdown.value)



unposed_selector.observe(render, 'value')
posed_selector.observe(render, 'value')
interval_dropdown.observe(render, 'value')

posed_selector.observe(set_img_sliders, 'value')
unposed_selector.observe(set_img_sliders, 'value')





# image_names = os.listdir(os.path.join(data_root, '0-calibration', interval_dropdown.value, 'images'))

# for image_name in image_names:
#     if image_name.split('_')[0] in unposed_cameras:
#         unposed_cameras[image_name.split('_')[0]].add_image(image_name)
#     else:
#         posed_cameras[image_name.split('_')[0]].add_image(image_name)


SelectMultiple(description='Unposed Cameras', options=('cam3',), value=())

SelectMultiple(description='Posed Cameras', options=('cam360', 'cam2', 'cam1'), value=())